In [1]:
import json 
fp = open("./dialog.json")
file = json.load(fp)

In [12]:
import os
import openai
openai.api_key = "sk-"
from eval_F1 import exact_match_score, f1_score, metric_max_over_ground_truths

def prepare_prompt(context):
    turns = len(context)
    dialog_text =""
    for i in range(turns):
        dialog_text += "\nUser1: " + context[i]["user1"] + "\nUser2: "+context[i]["user2"]
    final_prompt = "You are given a dialog between user1 and user2.  You need to identify the current entity being discussed.\n"+ dialog_text + "\n\nEntity: "
    return final_prompt

def get_gpt_response(text:str):
    response = openai.Completion.create(
    model="text-davinci-003",
    prompt=text,
    temperature=0.5,
    max_tokens=500,
    top_p=1.0,
    frequency_penalty=0.8,
    presence_penalty=0.0
    )
    return response["choices"][0]["text"]

def get_result_pipeline(context) -> str:
    prompt = prepare_prompt(context)
    return get_gpt_response(prompt)

def get_em(predict,gt):
    return metric_max_over_ground_truths(exact_match_score, predict, [gt])

def get_f1(predict,gt):
    return metric_max_over_ground_truths(f1_score, predict, [gt])

def main_result(file):
    length = len(file['dialog'])
    for i in range(length):
        print(f"Turn {i}")
        context = file['dialog'][i]['context']
        predict_entity = get_result_pipeline(context)
        gt = file['dialog'][i]['entity']
        em_score = get_em(predict_entity,gt)
        f1_score = get_f1(predict_entity,gt)
        file['dialog'][i]['predict'] = predict_entity
        file['dialog'][i]['em_score'] = em_score
        file['dialog'][i]['f1_score'] = f1_score
    return file

In [ ]:
out_file = main_result(file)
out_path = "./gpt_result_all.json"
with open(out_path,'w') as f:
    json.dump(out_file,f)

In [5]:
import json 
fp = open("./gpt_result_all.json")
file = json.load(fp)

In [ ]:
file[1]

In [16]:
import json
import pandas as pd
import torch
from datasets import load_dataset
from torch.utils.data import Dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import evaluate
import os 
os.environ["WANDB_DISABLED"] = "true"
from transformers import (
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    default_data_collator,
)
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
rouge = evaluate.load("rouge")

def compute_metrics(eval_preds):
    labels_ids = eval_preds.label_ids
    pred_ids = eval_preds.predictions
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    result = rouge.compute(predictions=pred_str, references=label_str)
    return result

def prepare_prompt(context):
    turns = len(context)
    dialog_text =""
    for i in range(turns):
        dialog_text += "\nUser1: " + context[i]["user1"] + "\nUser2: "+context[i]["user2"]
    final_prompt = "You are given a dialog between user1 and user2.  You need to identify the current topic being discussed.\n"+ dialog_text + "\n\nTopic: "
    return final_prompt

class t5_dataset(Dataset):
    def __init__(self, train_path, tokenizer, max_length=300,is_eval = False):
        fp = open(train_path)
        self.dataset = json.load(fp)
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.is_valid = is_eval 
        if(self.is_valid == False):
            self.input_ids = [torch.tensor(self.tokenizer(prepare_prompt(self.dataset[i]['context']),truncation=True, max_length=300, padding="max_length")['input_ids']) for i in range(1000,len(self.dataset))]
            self.atten_masks = [torch.tensor(self.tokenizer(prepare_prompt(self.dataset[i]['context']),truncation=True, max_length=300, padding="max_length")['attention_mask']) for i in range(1000,len(self.dataset))]
            self.labels = [torch.tensor(self.tokenizer(self.dataset[i]['predict'],truncation = True,max_length = 10,padding="max_length")["input_ids"]) for i in range(1000,len(self.dataset))]
    def __len__(self):
        if(self.is_valid == False):
            return len(self.dataset) - 1000
        else:
            return 1000
    
    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.atten_masks[idx],
            "labels": self.labels[idx],
        }

In [17]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
ds_train = t5_dataset("./gpt_result_all.json",tokenizer)
ds_eval = t5_dataset("./gpt_result_all.json",tokenizer,is_eval= True)

In [18]:
from transformers import pipeline
pipe = pipeline("text2text-generation",model = "google/flan-t5-small")

In [23]:
pid = 60
print(tokenizer.decode(ds_train[pid]['input_ids'],skip_special_tokens=True))
print(tokenizer.decode(ds_train[pid]['labels'],skip_special_tokens=True))

You are given a dialog between user1 and user2. You need to identify the current topic being discussed. User1: Hey! Ever play badminton? User2: I did in high school once. Crazy to think it's an Olympic sport! User1: It is? oh wow I didn't even think that was possible haha, never thought of it as much of a sport User2: Yeah, you should watch it on YouTube. I think they only play doubles in the Olympics, although badminton can be played as singles as well. User1: I'll definitely check up on that! Is it as entertaining as tennis? User2: It's definitely quicker than tennis! Tennis is fast and hard, but badminton is played so quickly that the time between each side's strikes is less than a second! User1: Makes sense, I remember how aerodynamic the ball (?) in badminton is. What is that thing called again? User2: Great question! It's called a shuttlecock, which is a ball with feathers attached to it to create drag. Topic: 
Shuttlecock


In [24]:
pipe(tokenizer.decode(ds_train[pid]['input_ids'],skip_special_tokens=False))

[{'generated_text': 'Badminton'}]

In [38]:

training_args = TrainingArguments(
    output_dir="./out",
    learning_rate=1e-5,
    evaluation_strategy="steps",
    eval_accumulation_steps=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    gradient_checkpointing=False,
    gradient_accumulation_steps=5,
    num_train_epochs=2,
    warmup_steps=100,
    save_steps=50,
    eval_steps=10,
    load_best_model_at_end=True,
    logging_steps=50,
)



PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [39]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_eval,
    data_collator=default_data_collator,
    compute_metrics = compute_metrics,
)
trainer.train()

Step,Training Loss,Validation Loss


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│    6 │   data_collator=default_data_collator,                                                    │
│    7 │   compute_metrics = compute_metrics,                                                      │
│    8 )                                                                                           │
│ ❱  9 trainer.train()                                                                             │
│   10                                                                                             │
│                                                                                                  │
│ /home/wentaoy4/anaconda3/envs/trlx/lib/python3.9/site-packages/transformers/trainer.py:1543 in   │
│ train                                                                                            │
│                                                                                                  │
│   1540 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1541 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1542 │   │   )                                                                                 │
│ ❱ 1543 │   │   return inner_training_loop(                                                       │
│   1544 │   │   │   args=args,                                                                    │
│   1545 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1546 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /home/wentaoy4/anaconda3/envs/trlx/lib/python3.9/site-packages/transformers/trainer.py:1791 in   │
│ _inner_training_loop                                                                             │
│                                                                                                  │
│   1788 │   │   │   │   │   with model.no_sync():                                                 │
│   1789 │   │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                  │
│   1790 │   │   │   │   else:                                                                     │
│ ❱ 1791 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1792 │   │   │   │                                                                             │
│   1793 │   │   │   │   if (                                                                      │
│   1794 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /home/wentaoy4/anaconda3/envs/trlx/lib/python3.9/site-packages/transformers/trainer.py:2557 in   │
│ training_step                                                                                    │
│                                                                                                  │
│   2554 │   │   │   # loss gets scaled under gradient_accumulation_steps in deepspeed             │
│   2555 │   │   │   loss = self.deepspeed.backward(loss)                                          │
│   2556 │   │   else:                                                                             │
│ ❱ 2557 │   │   │   loss.backward()                                                               │
│   2558 │   │                                                                                     │
│   2559 │   │   return loss.detach()                        